In [ ]:
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask", 
    model="hunarbatra/CoVBERT",
    tokenizer="hunarbatra/CoVBERT"
)

In [ ]:
mask_filler("MAT<mask>ALM")

[{'score': 0.6879484057426453,
  'token': 42,
  'token_str': 'F',
  'sequence': 'MATFALM'},
 {'score': 0.07334183901548386,
  'token': 50,
  'token_str': 'N',
  'sequence': 'MATNALM'},
 {'score': 0.041896820068359375,
  'token': 47,
  'token_str': 'K',
  'sequence': 'MATKALM'},
 {'score': 0.027228692546486855,
  'token': 61,
  'token_str': 'Y',
  'sequence': 'MATYALM'},
 {'score': 0.026297230273485184,
  'token': 43,
  'token_str': 'G',
  'sequence': 'MATGALM'}]

# Variant Mutation Processing

In [ ]:
import pandas as pd
import math
from statistics import mean

In [ ]:
def isNaN(num):
    return num != num

In [ ]:
mutations = ['D614G', 'D950N', 'E484K', 'H655Y', 'H655Y', 'K417N', 'N501Y', 'P681H', 'Q493R']

# Common Variant Mutations Masking Testing

In [ ]:
mutations_seqs = {}
[mutations_seqs.setdefault(name, []) for name in mutations]

mutations_seqs_results = {}
[mutations_seqs_results.setdefault(name, []) for name in mutations]

mutations_seqs_original = {}
[mutations_seqs_original.setdefault(name, []) for name in mutations]

[[], [], [], [], [], [], [], [], []]

In [ ]:
line_count = 0

with open('validation-set.txt', 'r') as file:
    for line in file:
        line_count += 1
        print(line_count)
        for mutation in mutations:
            original = mutation[0]
            change = mutation[-1]
            position = int("".join(mutation[1:-1]))-1

            if position < len(line) and line[position] == original:
                mutations_seqs[mutation].append(line)
                sequence = line[:position] + '<mask>' + line[position+1:]
                curr_results = mask_filler(sequence)

                curr_sequence_pred = 0
                curr_sequence_nochange = 0

                for i in range(0, len(curr_results)):
                    if curr_results[i]['token_str'] == change:
                        curr_sequence_pred = curr_results[i]['score']
                    elif curr_results[i]['token_str'] == original:
                        curr_sequence_nochange = curr_results[i]['score']
                    if curr_sequence_pred > 0 and curr_sequence_nochange > 0:
                        break
                    
                mutations_seqs_results[mutation].append(curr_sequence_pred)
                mutations_seqs_original[mutation].append(curr_sequence_nochange)

In [ ]:
mutations_seqs_results_mean = {}
[mutations_seqs_results_mean.setdefault(name, 0) for name in mutations]

mutations_seqs_original_mean = {}
[mutations_seqs_original_mean.setdefault(name, 0) for name in mutations]

for mutation in mutations_seqs_original:
    curr_list = mutations_seqs_original[mutation]
    mutations_seqs_original_mean[mutation] = round(mean(curr_list), 2)

for mutation in mutations_seqs_results:
    curr_list = mutations_seqs_results[mutation]
    mutations_seqs_results_mean[mutation] = round(mean(curr_list), 2)

In [ ]:
mutations_seqs_results_mean

In [ ]:
mutations_seqs_original_mean

In [ ]:
for mutation in mutations_seqs_results:
    print(len(mutations_seqs_results[mutation]))

In [ ]:
for mutation in mutations_seqs_results:
    print(mutation + ' : ' + str(max(mutations_seqs_results[mutation])))